# Creating CNN

In [27]:
# import custom modules
import sys
sys.path.insert(0, "../src/util")
sys.path.insert(0, "../src/model")
sys.path.insert(0, "../src/data_util")

import json

from baseline import *

# imports for model
import torch
import torchvision
import os
import torch.optim as optim
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint as cp
from torchvision import datasets, models, transforms
from sklearn.metrics import f1_score

In [28]:
# load in configs
print('---> Running test target...')
with open('../config/data-params.json') as fh:
    data_cfg = json.load(fh)
    print('---> loaded data config')

with open('../config/model-params.json') as fh:
    model_cfg = json.load(fh)
    print('---> loaded model config')

---> Running test target...
---> loaded data config
---> loaded model config


In [29]:
# create dataloaders
dataloaders_dict, num_classes = create_dataloaders(
    "../" + data_cfg['dataDir'],
    model_cfg['batchSize'],
    model_cfg['inputSize']
)

---> Begin dataloader creation...
---> Finished creating dataloaders.


In [30]:
# Detect if we have a GPU available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [31]:
# Initialize the model for this run
model_ft, input_size = initialize_model(
    model_cfg['modelName'],
    num_classes,
    feature_extract = model_cfg['featureExtract'],
    use_pretrained=True
)

---> Begin model initialization...
---> Finished model initialization.


In [32]:
# send model to gpu
model_ft = model_ft.to(device)
params_update = params_to_update(model_ft, model_cfg['featureExtract'])

# Optimizer
optimizer_ft = optim.Adam(params_update, lr=model_cfg['lr'])

# Loss function
criterion = nn.CrossEntropyLoss()

In [ ]:
# train model
model_ft, loss_train, acc_train, fs_train, loss_val, acc_val, fs_val = train_model(
    model_ft,
    dataloaders_dict,
    criterion,
    optimizer = optimizer_ft,
    num_epochs = 1 #model_cfg['nEpochs']
)

In [33]:
# if already trained model load this
model_weights = torch.load("../" + data_cfg['hierarchyModelPath'])
model_ft.load_state_dict(model_weights)

<All keys matched successfully>

In [34]:
from PIL import Image

def imshow(inp):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    plt.pause(0.001)

In [ ]:
def plot_metric(metric_train, metric_val, title):
    fig, (ax) = plt.subplots(1, 1)
    fig.suptitle(title)
    ax.set(xlabel='epoch')
    ax.plot(metric_train, label='Training')
    ax.plot(metric_val, label='Validation')
    ax.legend(loc='upper left')
    
plot_metric(loss_train, loss_val, 'Loss')
plot_metric(acc_train, acc_val, 'Accuracy')
plot_metric(fs_train, fs_val, 'F-Score')